### Imports and functions

In [2]:
# !pip install pandas
# !pip install numpy
# !pip install dexom-python
# !pip install jproperties
# !pip install . --users
import pandas as pd
import numpy as np
import os
import glob
from cobra import io
from jproperties import Properties
from mana import modelling, batchs, results_processing, dars


c:\Users\louison.fresnais\AppData\Local\miniconda3\envs\cobrapy\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load properties file

In [3]:
props = Properties()
try:
    with open('props.properties', 'rb') as config_file:
        props.load(config_file)
except FileNotFoundError as e:
    print(e)
    print("\033[91m\033[1m "+"You must provide a props.properties file"+" \033[0m\033[91m")

### Load required datasets

In [4]:
#Define paths 
### Load the model 
model = io.load_json_model(props.get("modelFile").data)
if len(props.get("modelId").data) > 0:
    model.id = str(props.get("modelId").data)
# model.id = 'recon2.2'

### Load the metadata file
pheno = pd.read_csv(props.get("pheno").data,sep="\t",index_col=0)

### Get relevant data and process it
#If cpds is empty, ask the user a list of compounds
cpds = str(props.get("cpds").data).split('/')
if len(cpds) == 0:
    input_buf = ''
    while input_buf.lower() != 'stop':
        input_buf = input('Provide either a molecule name or stop')
        if input_buf.lower() != 'stop':
            cpds.append(input_buf)

#load barcode processed data
gprs = modelling.get_GPR_reactions(model)
hgnc_data = pd.read_csv(props.get("mappingFile").data, sep="\t", dtype='unicode')
data = pd.read_csv(props.get("data").data,sep="\t")

100% |########################################################################|


### Prepare datasets and working env

In [4]:
#map gene expression identifiers and build sub dataset
col_to_add = modelling.identify_model_gene_ids(model)
if col_to_add == "model not implemented":
    print("\033[91m\033[1m "+"Unknown model"+" \033[0m\033[91m")
data = modelling.map_single_column(data,hgnc_data,col_to_add)
barcodes_to_keep = {}
subset_data = pd.DataFrame()
anot_cols = ['PROBEID','SYMBOL','HGNC ID','ENTREZID','GENENAME','N_GENES_IDENTICAL_PROBE']
#build dir and go into the relevant dir
dirname = str(props.get("working_path").data)+str(props.get("dose").data).lower()+'_'+str(props.get("time").data).replace(' ','_')
if dirname.split("/")[-1] not in os.listdir(str(props.get("working_path").data)):
    os.mkdir(dirname)
os.chdir(dirname)
working_folders = ['batch_full/batch','csvs','full_rxn_enum_set','log_dir','working_full']
for name in working_folders:
    if name.split('/')[0] not in os.listdir():
        if name == "batch_full/batch":
            os.mkdir(name.split('/')[0])
            os.mkdir(name)
        else:
            os.mkdir(name)
            

for cpd in str(props.get("cpds").data).split("/"):
    barcodes_to_keep[cpd] = list(pheno[(pheno['dose_level'] == props.get("dose").data) & \
                                       (pheno['sacri_period'] == props.get("time").data) & (pheno['compound_name'] == cpd)].index)
    if subset_data.shape[0] == 0:
        subset_data = pd.DataFrame(data.loc[:,anot_cols+barcodes_to_keep[cpd]])
    else:
        subset_data = pd.concat([subset_data,pd.DataFrame(data.loc[:,barcodes_to_keep[cpd]])],axis=1)
#extract data and binarize with the 75/25 method
modelling.preprocess_data(subset_data,col_to_add,model,pickle=False)

100% |########################################################################|
100% |########################################################################|
100% |########################################################################|


### Write reaction enum batch scripts

In [5]:
for file in os.listdir('csvs/'):
    if ".CEL" in file:
        batchs.write_rxn_enum_script(script_path=str(props.get("rxn_enum_script_path").data),batch_directory='batch_full',\
                                     output_directory='working_full',modelfile='../input_data/recon2v2_biomass_corrected.json',\
                                     weightfile='csvs/'+file,reactionFile="../input_data/recon2_2_reactions.csv", para_batchs=True)

### Launch all the batchs on the cluster
To run reaction enum on the cluster, you will need the dexom-python package with its dependencies installed (preferrably in a conda environment ) and CPLEX installed (to install locally on the genotoul cluster)

### Check that all the batchs are done. If necessary launch the launch_failed_batch_reaction_enum.sh file

In [6]:
#go into the relevant dir
dirname = str(props.get("working_path").data)+str(props.get("dose").data).lower()+'_'+str(props.get("time").data).replace(' ','_')
if os.path.basename(dirname) not in os.path.basename(os.getcwd()):
    #if yes then we are back at the root of the notebook, change that
    os.chdir(dirname)
results_processing.remove_done_batchs('batch_full/batch/','working_full/',relax_param=True)
if len(glob.glob('batch_full/batch/*.CEL*.sh',recursive=False))>0:
    print(len(glob.glob('batch_full/batch/*.CEL*.sh',recursive=False)),' batchs to relaunch')
    raise FileExistsError
else:
    print("All batchs have been processed")


All batchs have been processed


### Concatenate solutions in one csv file per biological condition and replicate

In [7]:
#go into the relevant dir
dirname = str(props.get("working_path").data)+str(props.get("dose").data).lower()+'_'+str(props.get("time").data).replace(' ','_')
if os.path.basename(dirname) not in os.path.basename(os.getcwd()):
    #if yes then we are back at the root of the notebook, change that
    os.chdir(dirname)
results_processing.concatenate_solutions("working_full","full_rxn_enum_set",col_index="",ncpus=2)

100%|██████████| 2/2 [00:54<00:00, 27.25s/it]

End of Queue


### Generate batch for diversity enum, starting from full reaction enum results

To launch the diversity enum pipeline, you must have finished the following steps:
* Full enumeration completed
* Proabilities computation on all the solutions

In [8]:
#go into the relevant dir
dirname = str(props.get("working_path").data)+str(props.get("dose").data).lower()+'_'+str(props.get("time").data).replace(' ','_')
if os.path.basename(dirname) not in os.path.basename(os.getcwd()):
    #if yes then we are back at the root of the notebook, change that
    os.chdir(dirname)
working_folders = ['batch_dexom/batch','prev_sol_dir','full_div_enum_set','working_divers']
for name in working_folders:
    if name.split('/')[0] not in os.listdir():
        if name == "batch_dexom/batch":
            os.mkdir(name.split('/')[0])
            os.mkdir(name)
        else:
            os.mkdir(name)
for file in os.listdir('csvs/'):
    batchs.write_div_enum_script(script_path=str(props.get("div_enum_script_path").data),batch_directory = 'batch_dexom',
                                 rxn_enum_set_dir = 'full_rxn_enum_set',output_directory = 'working_divers',
                                   modelfile = '../input_data/recon2v2_biomass_corrected.json',weightfile = 'csvs/'+file,
                                     reactionFile = '../input_data/recon2_2_reactions.csv')

### Launch all the batchs on the cluster
To run reaction enum on the cluster, you will need the dexom-python package with its dependencies installed (preferrably in a conda environment ) and CPLEX installed (to install locally on the genotoul cluster)

### Check that all the batchs are done. If necessary launch the launch_failed_batch_reaction_enum.sh file

In [9]:
#go into the relevant dir
dirname = str(props.get("working_path").data)+str(props.get("dose").data).lower()+'_'+str(props.get("time").data).replace(' ','_')
if os.path.basename(dirname) not in os.path.basename(os.getcwd()):
    #if yes then we are back at the root of the notebook, change that
    os.chdir(dirname)
results_processing.remove_done_batchs('batch_dexom/batch/','working_divers/',relax_param=False)
print(len(glob.glob('batch_dexom/batch/*.CEL*.sh',recursive=False)),' batchs to relaunch')

0  batchs to relaunch


### Concatenate solutions in one csv file per biological condition and replicate

In [10]:
#go into the relevant dir
dirname = str(props.get("working_path").data)+str(props.get("dose").data).lower()+'_'+str(props.get("time").data).replace(' ','_')
if os.path.basename(dirname) not in os.path.basename(os.getcwd()):
    #if yes then we are back at the root of the notebook, change that
    os.chdir(dirname)
results_processing.concatenate_solutions("working_divers","full_div_enum_set",col_index="",ncpus=2)

100%|██████████| 2/2 [00:50<00:00, 25.26s/it]

End of Queue


### Remove zero biomass solutions

In [11]:
#go into the relevant dir
dirname = str(props.get("working_path").data)+str(props.get("dose").data).lower()+'_'+str(props.get("time").data).replace(' ','_')
if os.path.basename(dirname) not in os.path.basename(os.getcwd()):
    #if yes then we are back at the root of the notebook, change that
    os.chdir(dirname)
results_processing.remove_zerobiomass_solutions('full_rxn_enum_set','../input_data/recon2_2_reactions.csv')
results_processing.remove_zerobiomass_solutions('full_div_enum_set','../input_data/recon2_2_reactions.csv')

### Concatenate the solutions

In [12]:
#go into the relevant dir
dirname = str(props.get("working_path").data)+str(props.get("dose").data).lower()+'_'+str(props.get("time").data).replace(' ','_')
if os.path.basename(dirname) not in os.path.basename(os.getcwd()):
    #if yes then we are back at the root of the notebook, change that
    os.chdir(dirname)
if "full_enum" not in os.listdir():
    os.mkdir("full_enum")
#this function is too complex : TODO recode
results_processing.concatenate_solutions('working_divers', 'full_enum',col_index="",single_csv=False,ncpus=2, restart=False, combine_r_d_enum=True)

100%|██████████| 2/2 [01:32<00:00, 46.32s/it]

End of Queue


### Compute activation frequencies for all full enum solution's file

In [7]:
os.chdir(props.get("working_path").data)

In [9]:
rList = list(pd.read_csv("../"+props.get("rListFile").data).iloc[:,0])
freq_table = pd.DataFrame()
for dir in glob.iglob('./**/full_enum',recursive=True):
    freq_table = pd.concat([freq_table,dars.calculate_frequencies_for_dir(dir,rList = rList)])
freq_table

,Barcode,13DAMPPOX,1a_24_25VITD2Hm,1a_24_25VITD3Hm,1a_25VITD2Hm,1a_25VITD3Hm,1PPDCRp,24_25VITD2Hm,24_25VITD3Hm,25VITD2Hm,...,FAOXC5030m,FAOXC6040m,FAOXC61_3Zm,FAOXC7050m,FAOXC8060m,FAOXC81_5Zm,FAOXC9070m,FAOXCPRIST1x,FAOXCPRIST2x,FAOXCPRIST3x
003016028014.CEL,003016028014.CEL,0.905092,0.0,0.0,0.0,0.0,0.840302,0.0,0.0,0.0,...,0.999897,0.878804,1.0,0.999897,0.895156,1.0,0.999897,0.976713,0.976713,0.976713
003016028015.CEL,003016028015.CEL,0.890194,0.0,0.0,0.0,0.0,0.841052,0.0,0.0,0.0,...,1.000000,0.914003,1.0,1.000000,0.957599,1.0,1.000000,0.973146,0.973146,0.973146
003016028020.CEL,003016028020.CEL,0.834338,0.0,0.0,0.0,0.0,0.859634,0.0,0.0,0.0,...,0.998708,0.858558,1.0,0.998708,0.887621,1.0,0.998708,0.948547,0.948547,0.948547
003016028021.CEL,003016028021.CEL,0.855815,0.0,0.0,0.0,0.0,0.900576,0.0,0.0,0.0,...,0.999091,0.847429,1.0,0.999091,0.875922,1.0,0.999091,0.957563,0.957563,0.957563


In [73]:
def calculate_frequencies(data, name):
    prob_vec = [sum(data.iloc[:, col])/data.shape[0] for col in range(data.shape[1])]
    return pd.Series(prob_vec, name=name)


def calculate_frequencies_for_dir(full_enum_path,rList,output_file= ""):
    csv_files = os.listdir(full_enum_path)
    prob_table = pd.DataFrame(np.zeros((1, len(rList))))
    prob_table = pd.concat([calculate_frequencies(pd.read_csv(full_enum_path+"/"+csv_file,index_col=0), csv_file.split('_')[0])
                            for csv_file in csv_files], axis=1).transpose()
    prob_table.insert(0,"Barcode",prob_table.index)
    rList.insert(0,"Barcode")
    prob_table.columns = rList
    if len(output_file)> 0:
         prob_table.to_csv(output_file)
    return prob_table

In [71]:
freq_table

,Barcode,13DAMPPOX,1a_24_25VITD2Hm,1a_24_25VITD3Hm,1a_25VITD2Hm,1a_25VITD3Hm,1PPDCRp,24_25VITD2Hm,24_25VITD3Hm,25VITD2Hm,...,FAOXC5030m,FAOXC6040m,FAOXC61_3Zm,FAOXC7050m,FAOXC8060m,FAOXC81_5Zm,FAOXC9070m,FAOXCPRIST1x,FAOXCPRIST2x,FAOXCPRIST3x
003016028014.CEL,003016028014.CEL,0.905092,0.0,0.0,0.0,0.0,0.840302,0.0,0.0,0.0,...,0.999897,0.878804,1.0,0.999897,0.895156,1.0,0.999897,0.976713,0.976713,0.976713
003016028015.CEL,003016028015.CEL,0.890194,0.0,0.0,0.0,0.0,0.841052,0.0,0.0,0.0,...,1.000000,0.914003,1.0,1.000000,0.957599,1.0,1.000000,0.973146,0.973146,0.973146
